In [1]:
import pandas as pd
from typing import List, Dict, Optional
import inspect

# This notebook is for cleaning the input dataset
* Removal of non-ASCII kw
* Removal of offencive kw and URLs
* Dropping of columns not used

In [2]:
def remove_non_ascii_kw(df: pd.DataFrame) -> pd.DataFrame:
    """
    Removes rows from a DataFrame where the "Keyword" column contains non-ASCII characters.
    
    Args:
        df (pandas.DataFrame): The DataFrame to process.
    
    Returns:
        pandas.DataFrame: The modified DataFrame with rows removed.
    """
    # Remove rows where "Keyword" contains non-ASCII characters
    df_start_len = len(df)
    df = df[df['Keyword'].apply(lambda x: all(ord(c) < 128 for c in x))]
    
    # Print some statistics
    print(f"df start length: {df_start_len}")
    print(f"df final length: {len(df)}")
    print(f"records removed: {df_start_len - len(df)}")
    
    # Return the modified DataFrame
    return df


In [3]:
def fix_nan(df: pd.DataFrame) -> pd.DataFrame:
    """
    Replaces NaN values in the "SERP features" and "CPC" columns of a pandas DataFrame with appropriate default values.
    
    Args:
        df (pandas.DataFrame): The DataFrame to process.
    
    Returns:
        pandas.DataFrame: The modified DataFrame with NaN values replaced.
    """
    # Count NaN values in "SERP features" and "CPC" columns
    serp_nan = df['SERP features'].isna().sum()
    cpc_nan = df['CPC'].isna().sum()
    
    # Replace NaN values with appropriate default values
    df['SERP features'] = df['SERP features'].fillna('None')
    df['CPC'] = df['CPC'].fillna(0)
    
    # Print some statistics
    print(f"SERP features NaN records replaced with 'None': {serp_nan}")
    print(f"CPC NaN records replaced with 0: {cpc_nan}")
    
    # Return the modified DataFrame
    return df

In [4]:
def remove_blocklist_kw(df: pd.DataFrame, kw_blocklist: List[str]) -> pd.DataFrame:
    """
    Removes rows from a pandas DataFrame where the "Keyword" column contains any of the keywords in a blocklist.

    Args:
        df (pandas.DataFrame): The DataFrame to process.
        kw_blocklist (List[str]): A list of keywords to remove from the DataFrame.

    Returns:
        pandas.DataFrame: The modified DataFrame with the specified keywords removed.
    """
    # Get the length of the input DataFrame
    df_start_len = len(df)

    # Create a regular expression pattern from the blocklist
    kw_pattern = '|'.join(kw_blocklist)

    # Create a mask for rows containing keywords in the blocklist
    kw_mask = df['Keyword'].str.contains(kw_pattern, case=False)

    # Use the mask to remove the specified keywords from the DataFrame
    df = df[~kw_mask]

    # Print some statistics
    print(f"df start length: {df_start_len}")
    print(f"df final length: {len(df)}")
    print(f"records removed: {df_start_len - len(df)}")

    # Return the modified DataFrame
    return df

In [5]:
def remove_blocklist_url(df: pd.DataFrame, url_blocklist: List[str]) -> pd.DataFrame:
    """
    Removes rows from a pandas DataFrame where the "URL" column contains any of the URLs in a blocklist.

    Args:
        df (pandas.DataFrame): The DataFrame to process.
        url_blocklist (List[str]): A list of URLs to remove from the DataFrame.

    Returns:
        pandas.DataFrame: The modified DataFrame with the specified URLs removed.
    """
    # Get the length of the input DataFrame
    df_start_len = len(df)

    # Create a regular expression pattern from the blocklist
    url_pattern = '|'.join(url_blocklist)

    # Create a mask for rows containing URLs in the blocklist
    url_mask = df['URL'].str.contains(url_pattern, case=False)

    # Use the mask to remove the specified URLs from the DataFrame
    df = df[~url_mask]

    # Print some statistics
    print(f"df start length: {df_start_len}")
    print(f"df final length: {len(df)}")
    print(f"records removed: {df_start_len - len(df)}")

    # Return the modified DataFrame
    return df

In [6]:
def show_df_info(df: pd.DataFrame) -> pd.DataFrame:
    """
    Prints information about a DataFrame, including column names, data types, and non-null counts.
    
    Args:
        df (pandas.DataFrame): The DataFrame to process.
        
    Returns:
        pandas.DataFrame: A DataFrame containing the column names, data types, and non-null counts, and null counts of the input DataFrame.
    """
    col_names = df.columns.to_list()
    col_dtypes = df.dtypes.to_list()
    non_null_counts = df.count().to_list()
    null_counts = df.isnull().sum().to_list()
    info_df = pd.DataFrame({'column_name': col_names, 'dtype': col_dtypes, 'non_null_count': non_null_counts, 'null_count': null_counts})

    caller_frame = inspect.currentframe().f_back
    df_name = [var_name for var_name, var_val in caller_frame.f_locals.items() if var_val is df][0]

    print(f"DataFrame '{df_name}' has {len(df)} rows and {len(df.columns)} columns.")
    print("Here is a summary of the column names, data types and null counts:")
    return info_df


In [7]:
df1 = pd.read_csv('data/webmd.csv')
df2 = pd.read_csv('data/thespruce.csv')
df3 = pd.read_csv('data/amazon.csv')
df = pd.concat([df1, df2, df3])
df = df.reset_index(drop=True)
# columns_to_drop = ["KD", "CPC", "Traffic", "Current position", "Current URL inside", "Updated"]
columns_to_drop = ["Current position", "Current URL inside", "Updated"]
df = df.drop(columns_to_drop, axis=1)
df = df.rename(columns={'Current URL': 'URL'})

In [8]:
show_df_info(df)

DataFrame 'df' has 90000 rows and 7 columns.
Here is a summary of the column names, data types and null counts:


,column_name,dtype,non_null_count,null_count
0,Keyword,object,90000,0
1,SERP features,object,89313,687
2,Volume,int64,90000,0
3,KD,int64,90000,0
4,CPC,float64,76574,13426
5,Traffic,int64,90000,0
6,URL,object,90000,0


In [9]:
df = remove_non_ascii_kw(df)

df start length: 90000
df final length: 89917
records removed: 83


In [10]:
kw_block_list = ["penis", "vagina", "sex", "slut", "dick", "fuck", "milf", "cum ", "cunt", "rape", "porn", "boner", "cock", "whore", "bitch", "futa", "wet dreams", "cowgirl position",  "reverse cowgirl", "girl squirt", "girls squirt", "squirting-orgasm", " squirt pee", "girl's body", "dildo", "butt-plug", "good head", "smells bad down there", "xxx", "wormwood", "fetish",  "vibrator", "sandalias de mujer"]

In [11]:
df = remove_blocklist_kw(df, kw_block_list)

df start length: 89917
df final length: 88842
records removed: 1075


In [12]:
url_blocklist = ["aws.amazon", "www.amazon.com/kindle-dbs", "squirting-orgasm"]

In [13]:
df = remove_blocklist_url(df, url_blocklist)

df start length: 88842
df final length: 88704
records removed: 138


In [14]:
df = fix_nan(df)

SERP features NaN records replaced with 'None': 602
CPC NaN records replaced with 0: 13180


In [15]:
show_df_info(df)

DataFrame 'df' has 88704 rows and 7 columns.
Here is a summary of the column names, data types and null counts:


,column_name,dtype,non_null_count,null_count
0,Keyword,object,88704,0
1,SERP features,object,88704,0
2,Volume,int64,88704,0
3,KD,int64,88704,0
4,CPC,float64,88704,0
5,Traffic,int64,88704,0
6,URL,object,88704,0


In [16]:
df.to_csv('data/combined.csv', index=False)

In [17]:
df_10_000 = df.sample(n=10000, random_state=42)

In [18]:
df_10_000.to_csv('data/kw_10_000.csv', index=False)

In [19]:
df_1_000 = df_10_000.sample(n=1000, random_state=42)

In [20]:
df_1_000.to_csv('data/kw_1_000.csv', index=False)